In [1]:
# imports
from pathlib import Path
from time import time

import numpy as np
import pandas as pd

from utils import (
    read_single_problem_from_path_as_adjacency,
    read_single_problem_from_path_as_sparse,
    read_single_problem_from_path_as_sparse_from_adjacency,
    order_nodes_in_descending_order,
    check_validity_for_adjacency,
    transform_node_clique_to_zero_one,
    transform_zero_one_clique_to_nodes
)
from constants import BASE_INSTANCES_FILES, OTHER_INSTANCES_FILES, OTHER_INSTANCES_BEST_KNOWN
from heuristics import (
    HeuristicRunner,
    descending_degree_glutonous_heuristic,
    dynamic_descending_degree_glutonous_heuristic,
    descending_degree_random_heuristic,
    multiple_descending_degree_random_heuristic
)

# Constants
ROOT_DIR = Path.cwd().parent
# Instances pathes
INSTANCES_DIR = ROOT_DIR / "instances"
BASE_INSTANCES_DIR = INSTANCES_DIR / "project_instances"
OTHER_INSTANCES_DIR = INSTANCES_DIR / "other_instances"


In [2]:
_, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "random-10.col"
)
print(graph)
clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
print(clique)
print("Is the clique valid ? ", check_validity_for_adjacency(graph, clique))


[[0 1 0 1 1 1 0 0 1 0]
 [1 0 1 0 1 0 1 1 1 1]
 [0 1 0 1 0 1 0 0 0 1]
 [1 0 1 0 0 0 0 0 1 0]
 [1 1 0 0 0 1 1 1 0 1]
 [1 0 1 0 1 0 1 0 1 0]
 [0 1 0 0 1 1 0 1 1 1]
 [0 1 0 0 1 0 1 0 0 1]
 [1 1 0 1 0 1 1 0 0 0]
 [0 1 1 0 1 0 1 1 0 0]]
[0 1 0 0 1 0 1 1 0 1]
Is the clique valid ?  True


In [3]:
# Check that clique representation utils work
number_of_nodes, _, graph, degrees = read_single_problem_from_path_as_adjacency(
    instance_path=BASE_INSTANCES_DIR / "random-10.col"
)
clique = descending_degree_glutonous_heuristic(graph=graph, degrees=degrees)
print(clique)
print(transform_zero_one_clique_to_nodes(clique))
print(transform_node_clique_to_zero_one(number_of_nodes, transform_zero_one_clique_to_nodes(clique)))

[0 1 0 0 1 0 1 1 0 1]
[1, 4, 6, 7, 9]
[0 1 0 0 1 0 1 1 0 1]


In [4]:
# Test on project instances
runner = HeuristicRunner(
    instances_dir=BASE_INSTANCES_DIR,
    instances=BASE_INSTANCES_FILES,
    methods=[
        (descending_degree_glutonous_heuristic, "desc_base"),
        (dynamic_descending_degree_glutonous_heuristic, "desc_dyn"),
        (descending_degree_random_heuristic, "desc_ran"),
        (multiple_descending_degree_random_heuristic, "mult_ran")
    ],
    known_bests={},
    display_time_details=False
)
runner.run_all_methods_on_all_instances()
runner.display_results()

          instance     method clique best  total(s)
0   brock200_2.col  desc_base      7   //  0.033909
1   brock200_2.col   desc_dyn      7   //  0.031914
2   brock200_2.col   desc_ran      9   //  0.032934
3   brock200_2.col   mult_ran      7   //  0.071836
4              ///        ///    ///  ///       ///
5    dsjc125.1.col  desc_base      4   //  0.004967
6    dsjc125.1.col   desc_dyn      4   //  0.004017
7    dsjc125.1.col   desc_ran      3   //  0.003997
8    dsjc125.1.col   mult_ran      4   //  0.022966
9              ///        ///    ///  ///       ///
10   random-10.col  desc_base      5   //  0.000998
11   random-10.col   desc_dyn      5   //  0.001001
12   random-10.col   desc_ran      3   //  0.001007
13   random-10.col   mult_ran      3   //  0.001966
14             ///        ///    ///  ///       ///
15  random-100.col  desc_base      8   //  0.010003
16  random-100.col   desc_dyn      8   //  0.007978
17  random-100.col   desc_ran      8   //  0.009973
18  random-1

In [5]:
# Test on other bigger instances.
runner = HeuristicRunner(
    instances_dir=OTHER_INSTANCES_DIR,
    instances=OTHER_INSTANCES_FILES,
    methods=[
        (descending_degree_glutonous_heuristic, "desc_base"),
        (dynamic_descending_degree_glutonous_heuristic, "desc_dyn"),
        (descending_degree_random_heuristic, "desc_ran"),
        (multiple_descending_degree_random_heuristic, "mult_ran")
    ],
    known_bests=OTHER_INSTANCES_BEST_KNOWN,
    display_time_details=False
)
runner.run_all_methods_on_all_instances()
runner.display_results()

              instance     method clique best  total(s)
0       brock800_4.txt  desc_base     14   26  0.644272
1       brock800_4.txt   desc_dyn     14   26  0.676319
2       brock800_4.txt   desc_ran     15   26   0.63184
3       brock800_4.txt   mult_ran     15   26   0.87098
4                  ///        ///    ///  ///       ///
5          C2000.5.txt  desc_base     10   16  3.118224
6          C2000.5.txt   desc_dyn     10   16  3.062845
7          C2000.5.txt   desc_ran     10   16  2.801503
8          C2000.5.txt   mult_ran     14   16  3.422372
9                  ///        ///    ///  ///       ///
10          C500.9.txt  desc_base     43   57  0.342087
11          C500.9.txt   desc_dyn     42   57  0.348643
12          C500.9.txt   desc_ran     43   57  0.352994
13          C500.9.txt   mult_ran     44   57   0.71064
14                 ///        ///    ///  ///       ///
15  gen400_p0.9_75.txt  desc_base     47   75  0.230386
16  gen400_p0.9_75.txt   desc_dyn     47   75  0

In [6]:
# Test different size of choices for the nodes for random heuristic
methods = [
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=2), f"ran_{2}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=3), f"ran_{3}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=4), f"ran_{4}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=5), f"ran_{5}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=6), f"ran_{6}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=7), f"ran_{7}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=8), f"ran_{8}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=9), f"ran_{9}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=10), f"ran_{10}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=11), f"ran_{11}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=12), f"ran_{12}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=13), f"ran_{13}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=14), f"ran_{14}"),
    (lambda *args, **kwargs: descending_degree_random_heuristic(*args, **kwargs, size_of_choice=15), f"ran_{15}"),
]
runner = HeuristicRunner(
    instances_dir=BASE_INSTANCES_DIR,
    instances=BASE_INSTANCES_FILES,
    methods=methods,
    known_bests={},
    display_time_details=False
)
runner.run_all_methods_on_all_instances()
runner.display_results()

          instance  method clique best  total(s)
0   brock200_2.col   ran_2      7   //  0.034906
1   brock200_2.col   ran_3      8   //  0.031942
2   brock200_2.col   ran_4      7   //  0.031914
3   brock200_2.col   ran_5      7   //  0.032882
4   brock200_2.col   ran_6      7   //  0.033295
..             ...     ...    ...  ...       ...
85   random-70.col  ran_12      5   //  0.004987
86   random-70.col  ran_13      7   //  0.004986
87   random-70.col  ran_14      5   //  0.005015
88   random-70.col  ran_15      7   //  0.005985
89             ///     ///    ///  ///       ///

[90 rows x 5 columns]
